In [29]:
import pandas as pd
import numpy as np
from auxiliary.preprocessing import *
from sklearn.model_selection import KFold, train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from tester import test_models_total

In [7]:
pd.set_option('display.max_rows', 200)
final_chosen = pd.read_csv('final.csv')

## SET Altman

In [8]:
var1 = ['A','B','C','D','E','Z']
data1 = final_chosen[['cik','fyear','ryear','at','act','csho','ebit','lct','lt',
                      're','revt','wcap','prcc_f','h1','h2','h3']].copy()
data1 = data1.dropna()
data1 = data1[(data1[['cik','fyear','ryear','at','act','csho','ebit','lct','lt',
                      're','revt','wcap','prcc_f']] != 0).all(axis=1)]
data1 = data1.reset_index(drop=True)
data1['A'] = data1['wcap'] / data1['at']
data1['B'] = data1['re'] / data1['at']
data1['C'] = data1['ebit'] / data1['at']
data1['D'] = data1['prcc_f'] * data1['csho'] / data1['lt']
data1['E'] = data1['revt'] / data1['at']
data1['Z'] = 1.2* data1['A'] + 1.4* data1['B'] + 3.3 * data1['C'] + 0.6 * data1['D'] + 1.0 * data1['E']
data1 = winsorize_df(data1,var1)
data1 = min_max_normalize(data1,var1)
data1

,cik,fyear,ryear,at,act,csho,ebit,lct,lt,re,...,prcc_f,h1,h2,h3,A,B,C,D,E,Z
0,1750.0,1996,1997,529.584,414.100,18.204,42.890,99.981,260.325,122.676,...,30.999985,-1,-1,-1,0.919079,0.783109,0.621534,0.084139,0.383066,0.263564
1,1750.0,1997,1998,670.559,468.400,27.704,64.716,149.148,369.709,147.590,...,26.437500,-1,-1,-1,0.769644,0.778262,0.655054,0.076506,0.403548,0.254183
2,1750.0,1998,1999,726.630,508.186,27.381,77.381,173.586,400.595,178.405,...,19.750000,-1,-1,-1,0.749706,0.788935,0.676610,0.050701,0.440658,0.240908
3,1750.0,1999,2000,740.998,511.267,26.865,70.658,163.816,401.483,201.026,...,13.875000,-1,-1,-1,0.760449,0.799753,0.652559,0.033468,0.486147,0.233778
4,1750.0,2000,2001,701.854,485.856,26.937,45.790,125.392,361.642,204.065,...,14.000000,-1,-1,-1,0.817509,0.807923,0.587530,0.038144,0.433859,0.228819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77837,1890567.0,2023,2023,78713.850,41510.140,1610.567,5579.784,25220.664,54148.656,7805.949,...,18.790000,-1,-1,-1,0.426007,0.727492,0.599721,0.018359,0.257646,0.150455
77838,1944057.0,2023,2023,2056.943,154.253,53.703,200.893,293.330,1250.363,172.960,...,12.720000,-1,-1,-1,0.075468,0.721160,0.657548,0.017846,0.027978,0.100624
77839,1900304.0,2022,2022,41948.594,4890.689,4617.287,2913.448,5265.413,22119.555,19566.371,...,8.000000,-1,-1,-1,0.150388,0.881679,0.596624,0.063785,0.076780,0.175879
77840,1900304.0,2023,2023,43379.258,6137.168,4617.287,3181.953,5910.432,22069.860,21035.533,...,8.230000,-1,-1,-1,0.168466,0.889439,0.605044,0.065906,0.084428,0.181998


## SET Barboza

In [9]:
var2 = ['A','B','C','D','E','OM','G_at','G_revt','G_emp','C_roe','C_pb']
data2 = final_chosen[['cik','fyear','ryear','at','act','ceq','csho','ebit','emp','lct','lt',
                      're','revt','wcap','prcc_f','h1','h2','h3']].copy()
data2 = data2.dropna()
data2 = data2[(data2[['cik','fyear','ryear','at','act','ceq','csho','ebit','emp','lct','lt',
                      're','revt','wcap','prcc_f']] != 0).all(axis=1)]
data2 = data2.reset_index(drop=True)
data2['A'] = data2['wcap'] / data2['at']
data2['B'] = data2['re'] / data2['at']
data2['C'] = data2['ebit'] / data2['at']
data2['D'] = data2['prcc_f'] * data2['csho'] / data2['lt']
data2['E'] = data2['revt'] / data2['at']
data2['OM'] = data2['ebit'] / data2['revt']
data2['roe'] = data2['revt'] / data2['ceq']
data2['pb'] = data2['prcc_f'] / data2['ceq']
data2 = calculate_growth(data2,'at')
data2 = calculate_growth(data2,'revt')
data2 = calculate_growth(data2,'emp')
data2 = calculate_change(data2, 'roe')
data2 = calculate_change(data2, 'pb')
inds = data2[(data2['fyear'] - data2['fyear'].shift(1) != 1) & (data2['cik'] == data2['cik'].shift(1))].index
data2 = data2.drop(inds)
data2 = data2.reset_index(drop=True)
data2 = data2.drop(0)
data2 = data2.reset_index(drop=True)
data2 = winsorize_df(data2, var2)
data2 = min_max_normalize(data2,var2)
data2

,cik,fyear,ryear,at,act,ceq,csho,ebit,emp,lct,...,D,E,OM,roe,pb,G_at,G_revt,G_emp,C_roe,C_pb
0,1750.0,1997,1998,670.559,468.400,300.850,27.704,64.716,2.700,149.148,...,0.076477,0.399306,0.749957,2.599711,0.087876,0.468322,0.478468,0.436083,0.529885,0.408977
1,1750.0,1998,1999,726.630,508.186,326.035,27.381,77.381,2.900,173.586,...,0.050535,0.436297,0.751202,2.815759,0.060576,0.389362,0.416684,0.349995,0.516428,0.408863
2,1750.0,1999,2000,740.998,511.267,339.515,26.865,70.658,2.900,163.816,...,0.033210,0.481641,0.738869,3.017048,0.040867,0.361751,0.393324,0.319864,0.515410,0.427981
3,1750.0,2000,2001,701.854,485.856,340.212,26.937,45.790,2.500,125.392,...,0.037911,0.429520,0.725494,2.569736,0.041151,0.330354,0.287657,0.263759,0.470641,0.478334
4,1750.0,2001,2002,710.199,436.656,310.235,31.870,4.711,2.200,150.464,...,0.032517,0.297522,0.689242,2.058830,0.036875,0.358341,0.238148,0.271053,0.466252,0.466852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73887,1890567.0,2023,2023,78713.850,41510.140,23880.986,1610.567,5579.784,104.416,25220.664,...,0.018021,0.253870,0.756061,2.586901,0.000787,0.418655,0.401970,0.318649,0.505112,0.477786
73888,1944057.0,2023,2023,2056.943,154.253,735.990,53.703,200.893,1.647,293.330,...,0.017504,0.024937,1.000000,0.514966,0.017283,0.000000,0.000000,0.000000,0.358504,0.519166
73889,1900304.0,2022,2022,41948.594,4890.689,19677.223,4617.287,2913.448,24.622,5265.413,...,0.063688,0.073583,0.862697,0.664870,0.000407,1.000000,1.000000,1.000000,0.511863,0.435116
73890,1900304.0,2023,2023,43379.258,6137.168,21152.723,4617.287,3181.953,25.408,5910.432,...,0.065821,0.081206,0.861351,0.680597,0.000389,0.367949,0.387130,0.332849,0.502602,0.477576


## SET Popular

In [11]:
var3 = ['CR','NPM','ROA','ROE','GM','ROCE','IC']
data3 = final_chosen[['cik','fyear','ryear','act','lct','ni','revt','at','ceq','gp','ppent','ebit','xint','h1','h2','h3']].copy()
data3 = data3.dropna()
data3 = data3[(data3[['cik','fyear','ryear','act','lct','ni','revt','at','ceq','gp','ppent','ebit','xint']] != 0).all(axis=1)]
data3 = data3.reset_index(drop = True)
data3['CR'] = data3['act'] / data3['lct']
data3['NPM'] = data3['ni'] / data3['revt']
data3['ROA'] = data3['ni'] / data3['at']
data3['ROE'] = data3['ni'] / data3['ceq']
data3['GM'] = data3['gp'] / data3['revt']
data3['ROCE'] = data3['ebit'] / (data3['ppent'] + data3['act'] - data3['lct'])
data3['IC'] = data3['ebit'] / data3['xint']
data3 = winsorize_df(data3, var3)
data3 = min_max_normalize(data3,var3)
data3

,cik,fyear,ryear,act,lct,ni,revt,at,ceq,gp,...,h1,h2,h3,CR,NPM,ROA,ROE,GM,ROCE,IC
0,1750.0,1996,1997,414.100,99.981,23.025,589.328,529.584,269.259,120.828,...,-1,-1,-1,0.661508,0.770053,0.709081,0.557718,0.222672,0.399976,0.131072
1,1750.0,1997,1998,468.400,149.148,35.657,782.123,670.559,300.850,162.689,...,-1,-1,-1,0.482664,0.775220,0.729057,0.573620,0.226368,0.429714,0.133331
2,1750.0,1998,1999,508.186,173.586,41.671,918.036,726.630,326.035,190.322,...,-1,-1,-1,0.444632,0.775063,0.737654,0.578096,0.225507,0.439010,0.131956
3,1750.0,1999,2000,511.267,163.816,35.163,1024.333,740.998,339.515,191.226,...,-1,-1,-1,0.479178,0.766294,0.717272,0.566416,0.199938,0.425838,0.126630
4,1750.0,2000,2001,485.856,125.392,18.531,874.255,701.854,340.212,160.444,...,-1,-1,-1,0.613802,0.755886,0.673910,0.542762,0.196019,0.391711,0.122361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84641,1900304.0,2023,2023,6137.168,5910.432,1336.216,14396.488,43379.258,21152.723,9260.526,...,-1,-1,-1,0.107191,0.812651,0.682973,0.546954,0.765779,1.000000,0.141416
84642,1948862.0,2021,2021,436.863,326.123,138.508,756.466,686.085,288.849,379.162,...,-1,-1,-1,0.160991,0.884210,1.000000,0.747533,0.589767,0.715807,0.726105
84643,1948862.0,2022,2022,614.904,286.396,122.801,780.538,981.551,283.784,375.569,...,-1,-1,-1,0.305217,0.863785,0.877233,0.724992,0.564904,0.513502,1.000000
84644,1995306.0,2022,2022,153.396,117.230,-41.248,558.225,576.112,239.806,139.903,...,-1,-1,-1,0.155443,0.680519,0.472041,0.433656,0.279179,0.342395,0.115156


## Set of All Features

In [18]:

data4 = final_chosen[['cik','fyear','ryear','at','act','ceq','cogs','csho','dp','dt','ebit','ebitda','emp',
                      'epspi','fincf','gp','icapt','intan','intpn','invt','ivncf','lct','lt','ni','oancf',
                      'pi','ppent','re','rect','revt','seq','txt','wcap','xint','prcc_f','xrd', 'h1','h2','h3']].copy()
data4 = data4[(data4[['cik','fyear','ryear','at','act','ceq','cogs','csho','dp','dt','ebit','ebitda','emp',
                      'epspi','fincf','gp','icapt','intan','intpn','invt','ivncf','lct','lt','ni','oancf',
                      'pi','ppent','re','rect','revt','seq','txt','wcap','xint','prcc_f','xrd']] != 0).all(axis=1)]
"""
data4 = final_chosen[['cik','fyear','ryear','at','act','ceq','cogs','csho','dp','dt','ebit','ebitda','emp',
                      'epspi','fincf','gp','icapt','intan','intpn','invt','ivncf','lct','lt','ni','oancf',
                      'pi','ppent','re','rect','revt','seq','txt','wcap','xint','prcc_f', 'h1']].copy()
data4 = data4[(data4[['cik','fyear','ryear','at','act','ceq','cogs','csho','dp','dt','ebit','ebitda','emp',
                      'epspi','fincf','gp','icapt','intan','intpn','invt','ivncf','lct','lt','ni','oancf',
                      'pi','ppent','re','rect','revt','seq','txt','wcap','xint','prcc_f']] != 0).all(axis=1)]
"""
data4 = data4.dropna()
data4 = data4.reset_index()
from sklearn.preprocessing import StandardScaler

data4['cem'] = data4['ppent'] + data4['act'] - data4['lct']
data4['cf'] = data4['oancf'] + data4['ivncf'] + data4['fincf']
data4['lnt'] = data4['lt'] - data4['lct']

data4 = calculate_avg(data4,'act')
data4 = calculate_avg(data4,'at')
data4 = calculate_avg(data4,'invt')
data4 = calculate_avg(data4,'ppent')

data4 = calculate_growth(data4,'revt')
data4 = calculate_growth(data4,'ni')
data4 = calculate_growth(data4,'at')
data4 = calculate_growth(data4,'emp')
data4 = calculate_growth(data4,'ebit')

data4 = calculate_change(data4,'ppent')
data4 = calculate_change(data4,'ceq')
data4 = calculate_change(data4,'wcap')

inds = data4[(data4['fyear'] - data4['fyear'].shift(1) != 1) & (data4['cik'] == data4['cik'].shift(1))].index
data4 = data4.drop(inds)
data4 = data4.reset_index(drop=True)
data4 = data4.drop(0)
data4 = data4.reset_index(drop=True)

# Altman
data4['A'] = data4['wcap'] / data4['at']
data4['B'] = data4['re'] / data4['at']
data4['C'] = data4['ebit'] / data4['at']
data4['D'] = data4['prcc_f'] * data4['csho'] / data4['lt']
data4['E'] = data4['revt'] / data4['at']
data4['Z'] = 1.2* data4['A'] + 1.4* data4['B'] + 3.3 * data4['C'] + 0.6 * data4['D'] + 1.0 * data4['E']
#Profitability
data4['NPM'] = data4['ni'] / data4['revt']
data4['ROA'] = data4['ni'] / data4['at']
data4['ROE'] = data4['ni'] / data4['ceq']
data4['GM'] = data4['gp'] / data4['revt']
data4['OM'] = data4['ebit'] / data4['revt']
data4['EBITDAM'] = data4['ebitda'] / data4['revt']
data4['OCFA'] = data4['oancf'] / data4['at']
data4['WCS'] = data4['wcap'] / data4['revt']
data4['ROCE'] = data4['ebit'] / data4['cem']
data4['CFR'] = data4['cf'] / data4['revt']
data4['OCFR'] = data4['oancf'] / data4['revt']
data4['ROIN'] = data4['revt'] / data4['invt']
data4['NPCA'] = data4['ni'] / data4['act']
data4['EBITDAA'] = data4['ebitda'] / data4['at']
data4['NPFA'] = data4['ni'] / data4['ppent']
data4['GOE'] = data4['gp'] / data4['ceq']
data4['PIOE'] = data4['pi'] / data4['ceq']
data4['ROIC'] = (data4['ebit'] - data4['txt']) / data4['icapt']
#Operating capacity
data4['CAT'] = data4['revt'] / data4['AVG_act']
data4['TAT'] = data4['revt'] / data4['AVG_at']
data4['CIR'] = data4['at'] / data4['revt']
data4['IT'] = data4['cogs'] / data4['AVG_invt']
data4['RDE'] = data4['G_revt'] / data4['xrd']
data4['FAT'] = data4['revt'] / data4['AVG_ppent']
#Solvency & Liquidity
data4['CR'] = data4['act'] / data4['lct']
data4['QR'] = (data4['act'] - data4['invt']) / data4['lct']
data4['SR'] = data4['ceq'] / data4['at']
data4['IC'] = data4['ebit'] / data4['xint']
data4['DOA'] = data4['dt'] / data4['ceq']
data4['G'] = data4['dt'] / data4['ceq']
data4['NPOL'] = data4['ni'] / data4['lt']
data4['EBITDAOL'] = data4['ebitda'] / data4['lt']
data4['FAOD'] = data4['ppent'] / data4['dt']
data4['LOFA'] = data4['lt'] / data4['ppent']
data4['OCFOL'] = data4['oancf'] / data4['lt']
data4['OCFOCL'] = data4['oancf'] / data4['lct']
#Structure
data4['WCOCA'] = data4['wcap'] / data4['act']
data4['CAOA'] = data4['act'] / data4['at']
data4['CEOFA'] = data4['cem'] / data4['ppent']
data4['FAOA'] = data4['ppent'] / data4['at']
data4['EOFA'] = data4['ceq'] / data4['ppent']
data4['CLOA'] = data4['lct'] / data4['at']
data4['CAIA'] = (data4['act'] - data4['invt']) / data4['at']
data4['AOL'] = data4['at'] / data4['lt']
data4['EOCL'] = data4['ceq'] / data4['lct']
data4['EONCL'] = data4['ceq'] / data4['lnt']
data4['EOL'] = data4['ceq'] / data4['lt']
data4['CLOL'] = data4['lct'] / data4['lt']
#Cash Flow
data4['CFOA'] = data4['cf'] / data4['at']
data4['CFOCL'] = data4['cf'] / data4['lct']
data4['CFOD'] = data4['cf'] / data4['dt']
#Growth
data4['CRR'] = (data4['C_ppent'] + data4['C_wcap']) / data4['cf']
data4['RDI'] = data4['xrd'] / data4['revt']
#Leverage
data4['OLC'] = data4['G_ebit'] / data4['G_revt']
data4['FLC'] = data4['ebit'] / (data4['ebit'] - data4['xint'])
data4['LC'] = data4['FLC'] * data4['OLC']
#Equity
data4['CFPS'] = data4['cf'] / data4['csho']
data4['PE'] = data4['prcc_f'] / data4['epspi']
data4['PCF'] = data4['prcc_f'] / data4['CFPS']
data4['PB'] = data4['prcc_f'] / data4['ceq']

var4 = [
    "A","B","C","D","E","Z",
    "NPM", "ROA", "ROE", "GM", "OM", "EBITDAM", "OCFA", "WCS", 
    "ROCE", "CFR", "OCFR", "ROIN", "NPCA", "EBITDAA", "NPFA", 
    "GOE", "PIOE", "ROIC", "CAT", "TAT", "CIR", "IT", "RDE", 
    "FAT", "CR", "QR", "SR", "IC", "DOA", "G", "NPOL", "EBITDAOL", 
    "FAOD", "LOFA", "OCFOL", "OCFOCL", "WCOCA", "CAOA", "CEOFA", 
    "FAOA", "EOFA", "C_ceq", "CLOA", "CAIA", "AOL", "EOCL", 
    "EONCL", "EOL", "CLOL", "CFOA", "CFOCL", "CFOD", "G_ni", 
    "CRR", "G_at", "G_revt", "G_emp", "RDI", "OLC", "LC", 
    "FLC", "epspi", "CFPS", "PE", "PCF", "PB"
]
"""
var4 = [
    "NPM", "ROA", "ROE", "GM", "OM", "EBITDAM", "OCFA", "WCS", 
    "ROCE", "CFR", "OCFR", "ROIN", "NPCA", "EBITDAA", "NPFA", 
    "GOE", "PIOE", "ROIC", "CAT", "TAT", "CIR", "IT", 
    "FAT", "CR", "QR", "SR", "IC", "DOA", "G", "NPOL", "EBITDAOL", 
    "FAOD", "LOFA", "OCFOL", "OCFOCL", "WCOCA", "CAOA", "CEOFA", 
    "FAOA", "EOFA", "C_ceq", "CLOA", "CAIA", "AOL", "EOCL", 
    "EONCL", "EOL", "CLOL", "CFOA", "CFOCL", "CFOD", "G_ni", 
    "CRR", "G_at", "G_revt", "G_emp", "OLC", "LC", 
    "FLC", "epspi", "CFPS", "PE", "PCF", "PB"
]
"""
data4 = winsorize_df(data4, var4)
data4 = min_max_normalize(data4,var4)
data4

,index,cik,fyear,ryear,at,act,ceq,cogs,csho,dp,...,CFOD,CRR,RDI,OLC,FLC,LC,CFPS,PE,PCF,PB
0,43,61478.0,2003,2003,1296.900,1006.000,627.700,422.600,806.600,59.200,...,0.426367,0.530461,0.422676,0.522176,0.458902,0.504129,0.490240,0.273806,0.489125,0.062472
1,44,61478.0,2004,2004,1428.100,835.600,659.300,440.700,810.100,41.700,...,0.154655,0.528686,0.225062,0.000000,0.555354,0.000000,0.398179,0.967385,0.480362,0.059876
2,45,61478.0,2005,2005,1535.000,853.000,773.900,681.100,116.500,67.200,...,0.267628,0.526504,0.182215,0.519099,0.512429,0.508587,0.475741,0.498218,0.514811,0.126995
3,46,61478.0,2006,2006,1611.400,942.700,873.500,801.100,117.200,67.800,...,0.259724,0.547772,0.167766,0.447120,0.545863,0.450450,0.457224,0.534807,0.525255,0.105437
4,47,61478.0,2007,2007,1764.800,1008.200,1007.600,827.500,117.600,67.200,...,0.399854,0.520502,0.156113,0.645402,0.525439,0.611864,0.779824,0.509325,0.489994,0.113047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17678,174515,1890567.0,2022,2022,68366.375,34596.254,21430.420,41874.390,1645.904,1963.278,...,0.236745,0.521396,0.092298,0.544171,0.492879,0.524164,0.330614,0.450146,0.473765,0.050676
17679,174516,1890567.0,2023,2023,78713.850,41510.140,23880.986,46487.510,1610.567,2142.160,...,0.255525,0.547470,0.091209,0.526263,0.488243,0.510734,0.524006,0.442203,0.501178,0.050907
17680,174520,1900304.0,2022,2022,41948.594,4890.689,19677.223,4629.226,4617.287,345.806,...,0.250672,0.000000,0.080195,0.485130,0.507245,0.482347,0.434700,0.550175,0.597260,0.049578
17681,174521,1900304.0,2023,2023,43379.258,6137.168,21152.723,5135.962,4617.287,393.604,...,0.254560,0.533683,0.079855,0.489396,0.525118,0.486196,0.441439,0.549197,0.537619,0.049516


## Set Variance Threshold

In [20]:
# Variance threshold
selector = VarianceThreshold(threshold=0.0566)
X_selected = selector.fit_transform(data4[var4])
print(X_selected.shape)
selector.get_feature_names_out()
data5 = final_chosen[['cik','fyear','ryear','at','gp','revt','ceq','wcap','act','ppent','lct','invt','lt','xrd', 'h1','h2','h3']].copy()
data5 = data5[(data5[['cik','fyear','ryear','at','gp','revt','ceq','wcap','act','ppent','lct','invt','lt','xrd']] != 0).all(axis=1)]

data5 = data5.dropna()
data5 = data5.reset_index()
var5 = list(selector.get_feature_names_out())
data5 = calculate_avg(data5,'at')
inds = data5[(data5['fyear'] - data5['fyear'].shift(1) != 1) & (data5['cik'] == data5['cik'].shift(1))].index
data5 = data5.drop(inds)
data5 = data5.reset_index(drop=True)
data5 = data5.drop(0)
data5 = data5.reset_index(drop=True)

# Altman
data5['A'] = data5['wcap'] / data5['at']
data5['E'] = data5['revt'] / data5['at']
#Profitability
data5['GM'] = data5['gp'] / data5['revt']
#Structure
data5['WCOCA'] = data5['wcap'] / data5['act']
data5['CAOA'] = data5['act'] / data5['at']
data5['FAOA'] = data5['ppent'] / data5['at']
data5['CLOA'] = data5['lct'] / data5['at']
data5['CAIA'] = (data5['act'] - data5['invt']) / data5['at']
data5['CLOL'] = data5['lct'] / data5['lt']
#Growth
data5['RDI'] = data5['xrd'] / data5['revt']

data5 = winsorize_df(data5, var5)
data5 = min_max_normalize(data5,var5)
data5[var5]

(17683, 9)


,A,E,GM,WCOCA,CAOA,FAOA,CLOA,CAIA,CLOL
0,0.610583,0.595998,0.549556,0.829739,0.598999,0.337443,0.266192,0.432494,1.000000
1,0.580288,0.492890,0.555548,0.736159,0.705476,0.284271,0.474499,0.605140,1.000000
2,0.580251,0.544374,0.571403,0.772148,0.638825,0.266442,0.374694,0.521695,0.990392
3,0.627855,0.361661,0.579172,0.779086,0.711318,0.212415,0.406675,0.629217,1.000000
4,0.468679,0.436742,0.398813,0.723791,0.509056,0.362232,0.359875,0.451962,1.000000
...,...,...,...,...,...,...,...,...,...
30521,0.141431,0.050955,0.748800,0.434256,0.000000,0.042014,0.116668,0.030814,0.467981
30522,0.091833,0.070587,0.736155,0.209902,0.000000,0.052239,0.121886,0.000000,0.121179
30523,0.110152,0.081864,0.732434,0.304430,0.000000,0.054357,0.144233,0.000000,0.156330
30524,0.312209,0.516444,0.550858,0.484675,0.668363,0.380109,0.850463,0.438434,0.856228


## Correlation Based Selection

In [21]:
#Corr
corr_matrix = data4[var4 + ['h3']].corr()
high_corr_features = corr_matrix.index[np.abs(corr_matrix["h3"]) > 0.0856]
print(high_corr_features)
data6 = final_chosen[['cik','fyear','ryear','re','oancf','wcap','csho','prcc_f','ebit','lt','ni','revt','at','act','ceq','epspi', 'h1','h2','h3']].copy()
data6 = data6[(data6[['cik','fyear','ryear','re','oancf','wcap','csho','prcc_f','ebit','lt','ni','revt','at','act','ceq','epspi']] != 0).all(axis=1)]

data6 = data6.dropna()
data6 = data6.reset_index()
var6 = list(high_corr_features)
var6.remove('h3')
# Altman
data6['A'] = data6['wcap'] / data6['at']
data6['B'] = data6['re'] / data6['at']
data6['C'] = data6['ebit'] / data6['at']
data6['D'] = data6['prcc_f'] * data6['csho'] / data6['lt']
data6['E'] = data6['revt'] / data6['at']
data6['Z'] = 1.2* data6['A'] + 1.4* data6['B'] + 3.3 * data6['C'] + 0.6 * data6['D'] + 1.0 * data6['E']
#Profitability
data6['NPM'] = data6['ni'] / data6['revt']
data6['ROA'] = data6['ni'] / data6['at']
data6['NPCA'] = data6['ni'] / data6['act']
data6['OCFA'] = data6['oancf'] / data6['at']
data6['OCFR'] = data6['oancf'] / data6['revt']
#Solvency & Liquidity
data6['SR'] = data6['ceq'] / data6['at']
data6['NPOL'] = data6['ni'] / data6['lt']
#Equity epspi

data6 = winsorize_df(data6, var6)
data6 = min_max_normalize(data6,var6)
data6[var6]

Index(['B', 'Z', 'NPM', 'ROA', 'OCFA', 'OCFR', 'NPCA', 'SR', 'NPOL', 'epspi',
       'h3'],
      dtype='object')


,B,Z,NPM,ROA,OCFA,OCFR,NPCA,SR,NPOL,epspi
0,0.783081,0.263381,0.810064,0.709412,0.370764,0.459328,0.587305,0.644569,0.559340,0.513678
1,0.778249,0.254022,0.814347,0.728095,0.412408,0.472907,0.598565,0.584885,0.564761,0.506839
2,0.788888,0.240778,0.814217,0.736135,0.425963,0.474881,0.601788,0.584924,0.569895,0.523556
3,0.799670,0.233664,0.806949,0.717072,0.359254,0.452689,0.594533,0.594400,0.558754,0.507599
4,0.807813,0.228716,0.798324,0.676517,0.494552,0.497481,0.577724,0.620905,0.534125,0.461246
...,...,...,...,...,...,...,...,...,...,...
76944,0.727645,0.150532,0.828768,0.727778,0.338110,0.449656,0.611951,0.439851,0.551622,0.602584
76945,0.721334,0.100817,1.000000,0.789303,0.575136,0.990319,1.000000,0.494182,0.594103,0.655775
76946,0.881329,0.175898,0.848527,0.684307,0.477891,0.640776,0.700082,0.605276,0.538530,0.430091
76947,0.889064,0.182003,0.845368,0.684994,0.484145,0.636402,0.676257,0.623790,0.540430,0.430851


## Mutual Information Based Selection

In [23]:
# Mutual Info
mi = mutual_info_classif(data4[var4], data4['h3'])
top_indices = np.argsort(mi)[-10:]

# Map the indices to feature names
selected_features = [var4[i] for i in top_indices]

# Select these features from the DataFrame
data4[selected_features]
var7 = ['D','NPFA','EOFA','EOL','AOL','DOA','SR','IC','FLC','PE']
data7 = final_chosen[['cik','fyear','ryear','prcc_f','csho','lt','ppent','ni','ceq','at','dt','ebit','xint','epspi', 'h1','h2','h3']].copy()
data7 = data7[(data7[['cik','fyear','ryear','prcc_f','csho','lt','ppent','ni','ceq','at','dt','ebit','xint','epspi']] != 0).all(axis=1)]

data7 = data7.dropna()
data7 = data7.reset_index()
#Altman
data7['D'] = data7['prcc_f'] * data7['csho'] / data7['lt']
#Profitability
data7['NPFA'] = data7['ni'] / data7['ppent']
#Structure
data7['EOFA'] = data7['ceq'] / data7['ppent']
data7['EOL'] = data7['ceq'] / data7['lt']
data7['AOL'] = data7['at'] / data7['lt']
#Solvency & Liquidity
data7['DOA'] = data7['dt'] / data7['ceq']
data7['SR'] = data7['ceq'] / data7['at']
data7['IC'] = data7['ebit'] / data7['xint']
#Growth
data7['FLC'] = data7['ebit'] / (data7['ebit'] - data7['xint'])
#Equity
data7['PE'] = data7['prcc_f'] / data7['epspi']

data7 = winsorize_df(data7, var7)
data7 = min_max_normalize(data7,var7)
data7[var7]

,D,NPFA,EOFA,EOL,AOL,DOA,SR,IC,FLC,PE
0,0.095198,0.463763,0.078199,0.230240,0.219662,0.368438,0.636184,0.197910,0.506388,0.502749
1,0.063953,0.454581,0.077202,0.237899,0.227181,0.366317,0.646138,0.188549,0.535202,0.489792
2,0.072430,0.437631,0.077978,0.260818,0.249683,0.366092,0.673979,0.181046,0.602192,0.541573
3,0.062702,0.353204,0.075909,0.221028,0.210618,0.382151,0.623739,0.165981,0.246433,0.402708
4,0.022257,0.409902,0.049915,0.215621,0.205310,0.389488,0.616182,0.165534,0.260560,0.370194
...,...,...,...,...,...,...,...,...,...,...
66345,0.118921,0.481625,0.191390,0.248487,0.239201,0.373757,0.657562,0.241394,0.474720,0.586165
66346,0.122767,0.480870,0.194103,0.265084,0.255552,0.369301,0.677009,0.214535,0.486604,0.585130
66347,0.205060,0.496760,0.046384,0.133922,0.129492,0.429810,0.468829,1.000000,0.456562,0.517377
66348,0.095379,0.987506,0.158862,0.080035,0.072551,0.341159,0.333358,0.193181,0.517734,0.486167


## Gradient Boosting Features Importances
Note that this method has randomness, so the features defined are the ones selected in a previous run

In [24]:
# Gradientboosting features importances
var8 = ['D','OCFR','GOE','FAT','CAT','EOFA','DOA','G','FLC','epspi']
data8 = final_chosen[['cik','fyear','ryear','prcc_f','csho','lt','oancf','revt','gp','ceq','ppent','act','dt','ebit','xint','epspi', 'h1','h2','h3']].copy()
data8 = data8[(data8[['cik','fyear','ryear','prcc_f','csho','lt','oancf','revt','gp','ceq','ppent','act','dt','ebit','xint','epspi']] != 0).all(axis=1)]

data8 = data8.dropna()
data8 = data8.reset_index()

data8 = calculate_avg(data8,'ppent')
data8 = calculate_avg(data8,'act')

inds = data8[(data8['fyear'] - data8['fyear'].shift(1) != 1) & (data8['cik'] == data8['cik'].shift(1))].index
data8 = data8.drop(inds)
data8 = data8.reset_index(drop=True)
data8 = data8.drop(0)
data8 = data8.reset_index(drop=True)

#Altman
data8['D'] = data8['prcc_f'] * data8['csho'] / data8['lt']
#Profitability
data8['OCFR'] = data8['oancf'] / data8['revt']
data8['GOE'] = data8['gp'] / data8['ceq']

#Operating capacity
data8['FAT'] = data8['revt'] / data8['AVG_ppent']
data8['CAT'] = data8['revt'] / data8['AVG_act']
#Structure
data8['EOFA'] = data8['ceq'] / data8['ppent']

#Solvency & Liquidity
data8['DOA'] = data8['dt'] / data8['ceq']
data8['G'] = data8['dt'] / data8['ceq']

#Leverage
data8['FLC'] = data8['ebit'] / (data8['ebit'] - data8['xint'])
#Growth

#Equity epspi


data8 = winsorize_df(data8, var8)
data8 = min_max_normalize(data8,var8)
data8[var8]

,D,OCFR,GOE,FAT,CAT,EOFA,DOA,G,FLC,epspi
0,0.060736,0.372416,0.392170,0.253980,0.220861,0.165663,0.403607,0.403607,0.554330,0.492978
1,0.069104,0.422395,0.375768,0.211127,0.189199,0.167321,0.403372,0.403372,0.627146,0.450140
2,0.059501,0.300237,0.356115,0.158483,0.143587,0.162899,0.420149,0.420149,0.240451,0.255618
3,0.019576,0.427707,0.357851,0.116911,0.152359,0.107329,0.427814,0.427814,0.255806,0.374298
4,0.048253,0.387021,0.367688,0.101088,0.166945,0.109408,0.431551,0.431551,1.000000,0.409410
...,...,...,...,...,...,...,...,...,...,...
57705,0.033692,0.369032,0.405959,0.118422,0.173110,0.105480,0.465233,0.465233,0.474857,0.580758
57706,0.032774,0.972301,0.364727,0.000000,0.000000,0.046287,0.479621,0.479621,0.567458,0.629916
57707,0.114998,0.582283,0.368252,0.164667,0.613756,0.409775,0.411380,0.411380,0.488589,0.421348
57708,0.118794,0.577402,0.369717,0.160431,0.295232,0.415576,0.406724,0.406724,0.501506,0.422051


## ANOVA Based Selection

In [25]:
#ANOVA
chi_scores = f_classif(data4[var4], data4['h3'])
top_indices = np.argsort(chi_scores[0])[-10:] 

# Map the indices to feature names
selected_features = [var4[i] for i in top_indices]
var10 = selected_features
data10 =   final_chosen[['cik','fyear','ryear','wcap','at','re','ebit','prcc_f','csho','lt','revt','oancf','ni','act','ceq','epspi', 'h1','h2','h3']].copy()
data10 = data10[(data10[['cik','fyear','ryear','wcap','at','re','ebit','prcc_f','csho','lt','revt','oancf','ni','act','ceq','epspi']] != 0).all(axis=1)]

data10 = data10.dropna()
data10 = data10.reset_index()

#Altman
data10['A'] = data10['wcap'] / data10['at']
data10['B'] = data10['re'] / data10['at']
data10['C'] = data10['ebit'] / data10['at']
data10['D'] = data10['prcc_f'] * data10['csho'] / data10['lt']
data10['E'] = data10['revt'] / data10['at']
data10['Z'] = 1.2* data10['A'] + 1.4* data10['B'] + 3.3 * data10['C'] + 0.6 * data10['D'] + 1.0 * data10['E']
#Profitability
data10['OCFR'] = data10['oancf'] / data10['revt']
data10['OCFA'] = data10['oancf'] / data10['at']
data10['NPM'] = data10['ni'] / data10['revt']
data10['ROA'] = data10['ni'] / data10['at']
data10['NPCA'] = data10['ni'] / data10['act']
#Operating capacity

#Structure

#Solvency & Liquidity
data10['NPOL'] = data10['ni'] / data10['lt']
data10['SR'] = data10['ceq'] / data10['at']
#Leverage

#Growth

#Equity epspi


data10 = winsorize_df(data10, var10)
data10 = min_max_normalize(data10,var10)
data10[var10]

,OCFR,Z,B,OCFA,NPOL,epspi,NPM,ROA,SR,NPCA
0,0.459328,0.263381,0.783081,0.370764,0.559340,0.513678,0.810064,0.709412,0.644569,0.587305
1,0.472907,0.254022,0.778249,0.412408,0.564761,0.506839,0.814347,0.728095,0.584885,0.598565
2,0.474881,0.240778,0.788888,0.425963,0.569895,0.523556,0.814217,0.736135,0.584924,0.601788
3,0.452689,0.233664,0.799670,0.359254,0.558754,0.507599,0.806949,0.717072,0.594400,0.594533
4,0.497481,0.228716,0.807813,0.494552,0.534125,0.461246,0.798324,0.676517,0.620905,0.577724
...,...,...,...,...,...,...,...,...,...,...
76944,0.449656,0.150532,0.727645,0.338110,0.551622,0.602584,0.828768,0.727778,0.439851,0.611951
76945,0.990319,0.100817,0.721334,0.575136,0.594103,0.655775,1.000000,0.789303,0.494182,1.000000
76946,0.640776,0.175898,0.881329,0.477891,0.538530,0.430091,0.848527,0.684307,0.605276,0.700082
76947,0.636402,0.182003,0.889064,0.484145,0.540430,0.430851,0.845368,0.684994,0.623790,0.676257


In [30]:
datas = [data1,data2, data3,data6,data7,data8,data10]
names = ['Altman','Barboza', 'Popular', 'Correlation Coefficient', 'Mutual Information', 'GDBT Features', 'ANOVA']
captions = ['Altman (1968)','Barboza (2017)', 'popular', 'CC', 'MI', 'GDBT-FI', 'ANOVA']

variables = [var1,var2,var3,var6,var7,var8,var10]

In [ ]:
results1 = test_models_total(datas,variables, captions, percentage = 0.1, k = 100, isTest = False,n_test = 5)
with open('variables.pkl', 'wb') as file:
    pickle.dump(results1, file)